In [46]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import glob

In [47]:
# import cv2
# import numpy as np

# # Load an image
# image = cv2.imread('images/CH1.jpg')

# # Convert the image from BGR to HSV color space
# hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

# # Define the lower and upper bounds for orange
# lower_banana = np.array([0, 50, 50])     # Lower bound for black
# upper_banana = np.array([40, 255, 255])   # Upper bound for yellow

# # Create a binary mask using the inRange function
# mask = cv2.inRange(hsv_image, lower_banana, upper_banana)

# # Find contours in the mask
# contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# # Iterate through contours
# for contour in contours:
#     # Get the bounding box of the contour
#     x, y, w, h = cv2.boundingRect(contour)

#     # Draw the bounding box on the original image
#     cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)

# # Display the result
# cv2.imshow('Bounding Box', image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [48]:
def get_background_color(image, region_of_interest):
    # Extract the region of interest (ROI) from the image
    roi = image[region_of_interest[1]:region_of_interest[3], region_of_interest[0]:region_of_interest[2]]

    # Calculate the mean color values in the ROI
    mean_color = np.mean(roi, axis=(0, 1))
    res = []
    for i in range (3):
        res.append(int(mean_color[i]))
    return res

In [49]:
# for pic_name in glob.glob(f"imgs/*.jpg"):
#     save_pic_name = pic_name.split("\\")[1].split('.')[0]
#     # Load an image
#     image = cv2.imread(pic_name)

#     # Convert the image from BGR to HSV color space
#     hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

#     # Define the lower and upper bounds for orange
#     lower_banana = np.array([0, 50, 50])     # Lower bound for black
#     upper_banana = np.array([40, 0, 255])   # Upper bound for yellow

#     # Create a binary mask using the inRange function
#     mask = cv2.inRange(hsv_image, lower_banana, upper_banana)

#     # Find contours in the mask
#     contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#     contours = sorted(contours, key=cv2.contourArea, reverse=True)[:3]
#     pad = 40

#     # Iterate through contours
#     for index, contour in enumerate(contours):
#         # Get the bounding box of the contour
#         x, y, w, h = cv2.boundingRect(contour)

#         # Draw the bounding box on the original image
#         cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)
#         # Draw the bounding box on the original image
#         # cv2.rectangle(image, (x - 100, y - 100), (x+1800, y+1800), (0, 255, 0), 2)
#         # Display the result
#         # cropped_image = image[y: y + h,x: x + w]
#         # roi_coordinates = (0, 0, 5, 5)
#         # background_color = get_background_color(cropped_image, roi_coordinates)
#         # max_width = max(w, h) + pad
#         # final_image = np.zeros((max_width, max_width, 3), dtype=np.uint8)
#         # cv2.rectangle(final_image, (0, 0), (max_width, max_width), background_color, -1)
#         # final_image[int((max_width-h)/2):int((max_width-h)/2) + h, int((max_width-w)/2):int((max_width-w)/2) + w] = cropped_image
#         # cv2.imwrite(f'cropped_imgs/{save_pic_name}_{index}.jpg', final_image)

#     # Display the result
#     cv2.imshow('Bounding Box', image)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()

#     # break

In [50]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import glob
import pandas as pd

df = pd.DataFrame()
for pic_name in glob.glob(f"0uM/*.jpg"):
    save_pic_name = pic_name.split("\\")[1].split('.')[0]
    # Load an image
    image = cv2.imread(pic_name)
    image = image[int(image.shape[0]/4) : int(image.shape[0]/2) + 1000, 
                  int(image.shape[1]/4) : int(image.shape[1]/2) + 300]
    # Convert BGR to HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Define range of orange color in HSV
    lower_orange = np.array([5,55,55])
    upper_orange = np.array([255, 255, 255])

    # Threshold the HSV image to get only orange colors
    mask = cv2.inRange(hsv, lower_orange, upper_orange)

    # Find contours
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # contour_image = cv2.drawContours(image.copy(), contours, -1, (0, 255, 0), 2)
    # Find the largest contour
    largest_contour = max(contours, key=cv2.contourArea)

    # # Create a mask with the largest contour filled
    mask_largest_contour = np.zeros_like(mask)
    cv2.drawContours(mask_largest_contour, [largest_contour], -1, 255, -1)

    # # Bitwise AND operation to extract the region of interest
    result = cv2.bitwise_and(image, image, mask=mask_largest_contour)

    # # Find bounding box coordinates
    x, y, w, h = cv2.boundingRect(largest_contour)

    # # Crop the extracted region
    cropped_contour = image[y:y+h, x:x+w]
    # cropped_contour = cv2.cvtColor(cropped_contour, cv2.COLOR_BGR2RGB)
    cv2.imwrite(f"0uM fn/{save_pic_name}.jpg",cropped_contour)
    color = get_background_color(cropped_contour, (int(w/2-5), int(h/2 - 5), int(w/2 + 5), int(h/2 + 5)))
    color.append(save_pic_name)
    df = pd.concat([df, pd.DataFrame([color])], ignore_index=True)
    # Display the result
    # cv2.imshow('Largest Contour Area', result)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    # plt.imshow(contour_image)
    # plt.show()
    
df.to_excel("res.xlsx")